# Final Composition
In this notebook I shall compose all the methods that I worked on so far and develop the final solution. A LOT of tinkering and moving around will be done, and some new features might be added, but I'll likely skip long sections for intermediate steps and directly include it in the document. 

In [43]:
# Set the right working folder to root folder of the project
import os

print("Looking for root folder of the project...")
for folder_depth in range(100): 
    if os.path.exists(".git"):
        root_folder = os.getcwd()
        print("Root folder found. Now working in directory '%s'" % os.getcwd())
        break
    else:
        print("Going up from '%s'" % os.getcwd())
        os.chdir("..")
else:
    raise Exception("Root folder of the project not found. Terminating.")
    

Looking for root folder of the project...
Root folder found. Now working in directory 'C:\Users\linas\projects\CarND-Advanced-Lane-Lines'


In [44]:
import glob
import matplotlib.image as mpimg
import matplotlib.pyplot as plt
%matplotlib inline

# Import everything needed to edit/save/watch video clips
from moviepy.editor import VideoFileClip
from moviepy.editor import clips_array
from moviepy.editor import ipython_display

import cv2
import numpy as np
from math import ceil


class Plotter:
    def __init__(self, columns, figsize=(20, 40)):
        plt.figure(figsize=figsize)
        
        self.columns = columns
        self.images = []
        self.extra_plots = []
        
    def add_img(self, img, title):
        assert len(img.shape) == 2 or img.shape[2] == 3
        
        cmap = None
        if len(img.shape) == 2:
            cmap = "gray"

        self.images.append((img, title, cmap))
        self.extra_plots.append([])
        
    def add_extra_to_last_img(self, xs, ys, plot_type):
        assert plot_type in ["line"]
        self.extra_plots[-1].append((xs, ys, plot_type))
            
    def plot(self):
        j = 1  # Current column
        i = 0
        rows = ceil(len(self.images) // self.columns)
        for img_index, (img, title, cmap) in enumerate(self.images):
            # Draw iamge
            ax = plt.subplot((rows / self.columns + 1) * self.columns, self.columns, i * self.columns + j)
            ax.set_title(title)
            
            # Draw Extras
            for xs, ys, plot_type in self.extra_plots[img_index]:
                if plot_type == "line":
                    plt.plot(xs, ys, color='yellow')
            
            # Show
            plt.imshow(img, cmap="gray")
            
            if j % self.columns == 0:
                j = 0
                i += 1
            j += 1


In [45]:
def apply_precomputed_undistortion(img, mtx_filename, dist_filename):
    mtx = np.load(mtx_filename)
    dist = np.load(dist_filename)
    undist_img = cv2.undistort(img, mtx, dist)
    return undist_img


def threshold_image(img):
    # Saturation-based thresholding
    hls_img = cv2.cvtColor(img, cv2.COLOR_RGB2HLS)
    s_channel = hls_img[:, :, 2]
    
    s_thresh_value = 150
    s_thresh = np.zeros_like(s_channel)
    s_thresh[s_channel > s_thresh_value] = 1
    
    # Edge Detection
    red = img[:, :, 0]
    sobel_kernel = 25
    assert sobel_kernel >= 3 and sobel_kernel % 2 == 1
    sobel_x = cv2.Sobel(red, cv2.CV_64F, 1, 0, ksize=sobel_kernel)
    sobel_y = cv2.Sobel(red, cv2.CV_64F, 0, 1, ksize=sobel_kernel)
    
    sobel_x = np.absolute(sobel_x)
    sobel_y = np.absolute(sobel_y)

    # Magnitude
    magnitude = (sobel_x**2 + sobel_y**2)**.5
    mag_norm = np.uint8(255 * magnitude / np.max(magnitude))
    mag_thresh_value = 30
    mag_thresh = np.zeros_like(mag_norm, dtype=np.uint8)
    mag_thresh[mag_norm > mag_thresh_value] = 1
    
    # Direction
    atan = np.arctan2(sobel_y, sobel_x)
    atan_thresh_min = 0.8
    atan_thresh_max = 1.2
    dir_thresh = np.zeros_like(atan, dtype=np.uint8)
    dir_thresh[(atan > atan_thresh_min) & (atan <= atan_thresh_max)] = 1
    
    # Combinations
    mag_and_dir = cv2.bitwise_and(mag_thresh, dir_thresh)    
    grad_or_color = cv2.bitwise_or(mag_and_dir, s_thresh)
    
    return grad_or_color


def warp_perspective(img, x_offset=200, y_offset=100, new_image_shape=(1000, 1000), flags=cv2.INTER_LINEAR):    
    # Detect Lines
    # Luckily, the camera resolution is 1280 x 720 in all iamges / videos we're given. This means I can hardode the values.
    top_left  = [578,  463]
    top_right = [706,  463]
    bot_right = [1043, 677]
    bot_left  = [267,  677]
    src_corners = np.float32([top_left, top_right, bot_right, bot_left])
    
    line_img = np.zeros(new_image_shape, dtype=np.uint8)
    
    new_top_left  = [x_offset, y_offset]
    new_top_right = [line_img.shape[1] - x_offset, y_offset]
    new_bot_right = [line_img.shape[1] - x_offset, line_img.shape[0] - y_offset]
    new_bot_left  = [x_offset, line_img.shape[0] - y_offset]
    dst_corners = np.float32([new_top_left, new_top_right, new_bot_right, new_bot_left])
    
    transform_matrix = cv2.getPerspectiveTransform(src_corners, dst_corners)
    inv_transform_matrix = cv2.getPerspectiveTransform(dst_corners, src_corners)
    warped = cv2.warpPerspective(img, transform_matrix, line_img.shape[::-1], flags=flags)

    return warped, transform_matrix, inv_transform_matrix


def retrieve_polylines(warped, draw_windows=True):    
    """
    Takes in the warped thresholds to find polylines on the road
    """
    
    # Find Curvature
    warped_y_midpoint = warped.shape[0] // 2
    histogram = np.sum(warped[warped_y_midpoint:, :], axis=0)
    out_img = np.dstack((warped, warped, warped))
    
    hist_x_midpoint = np.int(histogram.shape[0] // 2)
    left_x_base = np.argmax(histogram[:hist_x_midpoint])
    right_x_base = np.argmax(histogram[hist_x_midpoint:]) + hist_x_midpoint

    # Hyperparameters
    win_min_pix = 50  # minimum number of pixels found to recenter window
    win_num = 9  # the number of sliding windows
    win_half_width = 200 // 2
    win_height = np.int(warped.shape[0] // win_num)
    
    nonzero = warped.nonzero()
    nonzero_y = np.array(nonzero[0])
    nonzero_x = np.array(nonzero[1])
    
    left_x_current = left_x_base
    right_x_current = right_x_base
    left_lane_inds = []
    right_lane_inds = []
    
    for win_index in range(win_num):
        win_y_low = warped.shape[0] - (win_index + 1) * win_height
        win_y_high = warped.shape[0] - win_index * win_height
        win_x_left_low   = left_x_current - win_half_width
        win_x_left_high  = left_x_current  + win_half_width
        win_x_right_low  = right_x_current - win_half_width
        win_x_right_high = right_x_current + win_half_width
        
        # Draw the windows on the visualization image
        if draw_windows:
            cv2.rectangle(out_img, (win_x_left_low,  win_y_low), (win_x_left_high,  win_y_high), color=(0, 255, 0), thickness=3)
            cv2.rectangle(out_img, (win_x_right_low, win_y_low), (win_x_right_high, win_y_high), color=(0, 255, 0), thickness=3)
        
        good_left_inds = (
            (nonzero_x >= win_x_left_low) &
            (nonzero_x < win_x_left_high) &
            (nonzero_y >= win_y_low) &
            (nonzero_y < win_y_high)
        ).nonzero()[0]
        good_right_inds = (
            (nonzero_x >= win_x_right_low) &
            (nonzero_x < win_x_right_high) &
            (nonzero_y >= win_y_low) &
            (nonzero_y < win_y_high)
        ).nonzero()[0]
        
        left_lane_inds.append(good_left_inds)
        right_lane_inds.append(good_right_inds)
        
        if len(good_left_inds) > win_min_pix:
            left_x_current = np.int(np.mean(nonzero_x[good_left_inds]))
        if len(good_right_inds) > win_min_pix:        
            right_x_current = np.int(np.mean(nonzero_x[good_right_inds]))

    try:
        left_lane_inds = np.concatenate(left_lane_inds)
        right_lane_inds = np.concatenate(right_lane_inds)
    except ValueError as e:
        raise e

    left_x = nonzero_x[left_lane_inds]
    left_y = nonzero_y[left_lane_inds] 
    right_x = nonzero_x[right_lane_inds]
    right_y = nonzero_y[right_lane_inds]
    
    ## Visualization ##
    # Colors in the left and right lane regions
    out_img[left_y, left_x] = [255, 0, 0]
    out_img[right_y, right_x] = [0, 0, 255]
    
    # Polyfit
    left_fit = np.polyfit(left_y, left_x, 2)
    right_fit = np.polyfit(right_y, right_x, 2)

    return out_img, left_fit, right_fit


def retrieve_polylines_from_older(warped, left_fit, right_fit, draw_windows=True):
    """
    The same polyfit function, but this time it uses the window around previous polynomial lines to search for pixels.
    """
    # Hyperparameters
    win_half_width = 200 // 2

    nonzero = warped.nonzero()
    nonzero_y = np.array(nonzero[0])
    nonzero_x = np.array(nonzero[1])

    left_lane_inds = (
        (nonzero_x > (left_fit[0] * (nonzero_y**2) + left_fit[1] * nonzero_y + left_fit[2] - win_half_width)) &
        (nonzero_x < (left_fit[0] * (nonzero_y**2) + left_fit[1] * nonzero_y + left_fit[2] + win_half_width))
    )
    right_lane_inds = (
        (nonzero_x > (right_fit[0]*(nonzero_y**2) + right_fit[1] * nonzero_y + right_fit[2] - win_half_width)) &
        (nonzero_x < (right_fit[0]*(nonzero_y**2) + right_fit[1] * nonzero_y + right_fit[2] + win_half_width))
    )

    left_x = nonzero_x[left_lane_inds]
    left_y = nonzero_y[left_lane_inds] 
    right_x = nonzero_x[right_lane_inds]
    right_y = nonzero_y[right_lane_inds]

    left_fit = np.polyfit(left_y, left_x, 2)
    right_fit = np.polyfit(right_y, right_x, 2)
    plot_y = np.linspace(0, warped.shape[0] - 1, warped.shape[0])
    
    out_img = np.dstack((warped, warped, warped)) * 255
    win_img = np.zeros_like(out_img)

    out_img[nonzero_y[left_lane_inds],  nonzero_x[left_lane_inds]]  = [255, 0, 0]
    out_img[nonzero_y[right_lane_inds], nonzero_x[right_lane_inds]] = [0, 0, 255]

    left_line_window_1 = np.array([
        np.vstack([left_fitx - win_half_width, ploty]).T
    ])
    left_line_window_2 = np.array([
        np.flipud(np.vstack([left_fitx + win_half_width, ploty]).T)
    ])
    left_line_pts = np.hstack((left_line_window1, left_line_window2))
    
    right_line_window1 = np.array([
        np.vstack([right_fitx-win_half_width, ploty]).T
    ])
    right_line_window2 = np.array([
        np.flipud(np.vstack([right_fitx + win_half_width, ploty]).T)
    ])
    right_line_pts = np.hstack((right_line_window1, right_line_window2))

    cv2.fillPoly(win_img, np.int_([left_line_pts]), (0,255, 0))
    cv2.fillPoly(win_img, np.int_([right_line_pts]), (0,255, 0))
    result = cv2.addWeighted(out_img, 1, win_img, 0.3, 0)
    
    return result, left_fit, right_fit

def generate_polyline_plots(warped_shape, left_fit, right_fit):
    # Generate x and y values for plotting
    plot_y = np.linspace(0, warped_shape[0] - 1, warped_shape[0])

    left_fit_x = left_fit[0] * plot_y**2 + left_fit[1] * plot_y + left_fit[2]
    right_fit_x = right_fit[0] * plot_y**2 + right_fit[1] * plot_y + right_fit[2]

    # # Now call:
    # plotter.add_img(out_img, "warped image")
    # plotter.add_extra_to_last_img(left_fit_x,  plot_y, "line")
    # plotter.add_extra_to_last_img(right_fit_x, plot_y, "line")
    
    return left_fit_x, right_fit_x, plot_y
    
def fit_polynomial(x, coefficients):
    res = 0
    for power, coeff in enumerate(coefficients[::-1]):
        res += coeff * x**power
    return res

def get_lane_area(warped_line_img, inv_transformation_matrix, original_img_shape, left_fit, right_fit):
    # Area between two polylines
    all_pts = np.zeros(warped_line_img.shape[:2], dtype=np.uint8)
    for y in range(warped_line_img.shape[0]):
        poly_left = int(fit_polynomial(y, left_fit))
        poly_right = int(fit_polynomial(y, right_fit))
        all_pts[y, poly_left:poly_right] = 1
    
    polyfilled = warped_line_img.copy()
    polyfilled[all_pts == 1] = np.array([0, 255, 0])
    
    # Dewarp
    dewarped = cv2.warpPerspective(polyfilled, inv_transformation_matrix, original_img_shape[1::-1], flags=cv2.INTER_LINEAR)
    
    return dewarped

In [46]:
def threshold_yellow_lines(hls_img):
    """
    This function attempts to detect only the yellow lines.
    
    Why try to make a general function for detecting all lane lines, when we can easily distinguish two types - yellow and white?
    (actually, there's a third type, when road side is used as a line, with no explicit line marking)
    
    I have consulted https://driversed.com/driving-information/signs-signals-and-markings/markings-colors-patterns-meaning.aspx,
    verifying that only two color types exist.
    """
    # Convert to hls
    h_channel = hls_img[:, :, 0]
    l_channel = hls_img[:, :, 1]
    s_channel = hls_img[:, :, 2]
    
    thresh = np.zeros_like(h_channel)
    thresh[
        (h_channel >= 11) & (h_channel <= 30) &
        (s_channel >= 50) &
        (l_channel >= 150)
    ] = 255
    
    return thresh


def threshold_strong_sat_light(hls_img):
    h_channel = hls_img[:, :, 0]
    l_channel = hls_img[:, :, 1]
    s_channel = hls_img[:, :, 2]
    
    thresh = np.zeros_like(h_channel)
    thresh[
        (s_channel >= 150) &
        (l_channel >= 128)
    ] = 255

    return thresh


def apply_sobel(channel, kernel_size):
    assert kernel_size >= 3 and kernel_size % 2 == 1
    sobel_x = cv2.Sobel(channel, cv2.CV_64F, 1, 0, ksize=kernel_size)
    sobel_y = cv2.Sobel(channel, cv2.CV_64F, 0, 1, ksize=kernel_size)
    
    sobel_x = np.absolute(sobel_x)
    sobel_y = np.absolute(sobel_y)
    
    return sobel_x, sobel_y


def threshold_magnitude(sobel_x, sobel_y):
    magnitude = (sobel_x**2 + sobel_y**2)**.5
    mag_norm = np.uint8(255 * magnitude / np.max(magnitude))
    mag_thresh_value = 30
    mag_thresh = np.zeros_like(mag_norm, dtype=np.uint8)
    mag_thresh[mag_norm > mag_thresh_value] = 255
    
    return mag_thresh


def threshold_direction(sobel_x, sobel_y):
    atan = np.arctan2(sobel_y, sobel_x)  # returns results in range [0.0, 1.57079632679]
    atan_thresh_max = 1.2
    dir_thresh = np.zeros_like(atan, dtype=np.uint8)
    dir_thresh[(atan <= atan_thresh_max)] = 255
    
    return dir_thresh


def morph_close(thresh, kernel_size):
    assert kernel_size >= 3 and kernel_size % 2 == 1
    kernel = cv2.getStructuringElement(cv2.MORPH_ELLIPSE, (kernel_size, kernel_size))
    closed = cv2.morphologyEx(thresh, cv2.MORPH_CLOSE, kernel)
    
    return closed
    

In [47]:
# def process_img(img):
    
#     # Undistort Image
#     mtx_filename  = os.path.join(root_folder, "params", "camera_matrix.npy")
#     dist_filename = os.path.join(root_folder, "params", "dist_coeffs.npy")
#     undistorted = apply_precomputed_undistortion(img, mtx_filename, dist_filename)

#     # Threshold image
#     thresh = threshold_image(undistorted)

#     # Focus on the road lines by warping perspective
#     warped, transformation_matrix, inv_transformation_matrix = warp_perspective(thresh, x_offset=200, y_offset=100, new_image_shape=(1000, 1000))

#     # Fit a polyline
#     warped_line_img, left_fit, right_fit = retrieve_polylines(warped, draw_windows=False)

#     # Draw the area
#     dewarped = get_lane_area(warped_line_img, inv_transformation_matrix, img.shape, left_fit, right_fit)
    
#     # Overlay
#     alpha = 0.2
#     combined_img = np.zeros_like(img)
#     cv2.addWeighted(dewarped, alpha, img, 1 - alpha, 0, combined_img)   
    
#     return combined_img

def process_img(rgb_img):
    # Undistort Image
    mtx_filename  = os.path.join(root_folder, "params", "camera_matrix.npy")
    dist_filename = os.path.join(root_folder, "params", "dist_coeffs.npy")
    undistorted = apply_precomputed_undistortion(rgb_img, mtx_filename, dist_filename)
    
    # RGB -> HLS
    hls_img = cv2.cvtColor(undistorted, cv2.COLOR_RGB2HLS)
    h_channel = hls_img[:, :, 0]
    l_channel = hls_img[:, :, 1]
    s_channel = hls_img[:, :, 2]
    
    # Find x and y edges
    l_sobel_x, l_sobel_y = apply_sobel(l_channel, 25)
    s_sobel_x, s_sobel_y = apply_sobel(s_channel, 25)
    
    # Threshold by color
    thresh_yellow = threshold_yellow_lines(hls_img)
    thresh_sat_light = threshold_strong_sat_light(hls_img)

    # Threshold by edge magnitude
    l_thresh_mag = threshold_magnitude(l_sobel_x, l_sobel_y)
    s_thresh_mag = threshold_magnitude(s_sobel_x, s_sobel_y)
    thresh_mag = cv2.bitwise_or(l_thresh_mag, s_thresh_mag)
    
    # Threshold by edge direction
    l_thresh_dir = threshold_direction(l_sobel_x, l_sobel_y)  # I results are slightly better with just lightness-based threshold than both.
    
    # Combine edge magnitude and direction
    thresh_edges = cv2.bitwise_and(thresh_mag, l_thresh_dir)
    
    # Fill in gaps between edges
    closed_edges = morph_close(thresh_edges, 5)  
    
    result = closed_edges
    

    
#     # Focus on the road lines by warping perspective
#     warped, transformation_matrix, inv_transformation_matrix = warp_perspective(
#         undistorted, x_offset=200, y_offset=100, new_image_shape=img.shape[:2]
#     )
    
#     # Examine other color spaces
#     hls_warped = cv2.cvtColor(warped, cv2.COLOR_RGB2HLS)
#     one_channel = hls_warped[:, :, 1]
#     one_channel = np.uint8(one_channel / np.max(one_channel) * 255)
    
#     clahe = cv2.createCLAHE(clipLimit=2.0, tileGridSize=(8,8))
#     result = clahe.apply(one_channel)
    
    
#     plt.hist(reduced_hist)
#     plt.show()
    
#     one_channel = np.uint8(one_channel / np.max(one_channel) * 255)
    
#     s_thresh_value = 150
#     s_thresh = np.zeros_like(one_channel)
#     s_thresh[one_channel > s_thresh_value] = 1
    
#     s_thresh3 = cv2.cvtColor(s_thresh * 255, cv2.COLOR_GRAY2BGR)
    
    #     # Threshold image
#     thresh = threshold_image(undistorted)
        
    if len(np.shape(result)) == 2:
        return cv2.cvtColor(result, cv2.COLOR_GRAY2BGR)
    else:
        return result


In [48]:
video_in_folder  = os.path.join(root_folder, "data")
video_out_folder = os.path.join(root_folder, "results")
video_in_pattern = os.path.join(video_in_folder, "*.mp4")
video_in_fnames  = glob.glob(video_in_pattern)
video_out_fnames = [os.path.join(video_out_folder, os.path.basename(fname)) for fname in video_in_fnames]

only_use_videos = []  # Only use videos with indices specified here
use_subclip = False  # For testing

for i in range(len(video_in_fnames)):
    if only_use_videos and i not in only_use_videos:
        continue
    video_in_fname  = video_in_fnames[i]
    video_out_fname = video_out_fnames[i]
    
    clip = VideoFileClip(video_in_fname)
    if use_subclip:
        clip = clip.subclip(0, 10)
    
    processed_clip = clip.fl_image(process_img)

    %time processed_clip.write_videofile(video_out_fname, audio=False)
    
    clip.close()
    processed_clip.close()
    


[MoviePy] >>>> Building video C:\Users\linas\projects\CarND-Advanced-Lane-Lines\results\challenge_video.mp4
[MoviePy] Writing video C:\Users\linas\projects\CarND-Advanced-Lane-Lines\results\challenge_video.mp4


100%|██████████| 485/485 [02:15<00:00,  3.59it/s]


[MoviePy] Done.
[MoviePy] >>>> Video ready: C:\Users\linas\projects\CarND-Advanced-Lane-Lines\results\challenge_video.mp4 

Wall time: 2min 15s
[MoviePy] >>>> Building video C:\Users\linas\projects\CarND-Advanced-Lane-Lines\results\harder_challenge_video.mp4
[MoviePy] Writing video C:\Users\linas\projects\CarND-Advanced-Lane-Lines\results\harder_challenge_video.mp4


100%|█████████▉| 1199/1200 [05:35<00:00,  3.58it/s]


[MoviePy] Done.
[MoviePy] >>>> Video ready: C:\Users\linas\projects\CarND-Advanced-Lane-Lines\results\harder_challenge_video.mp4 

Wall time: 5min 36s
[MoviePy] >>>> Building video C:\Users\linas\projects\CarND-Advanced-Lane-Lines\results\project_video.mp4
[MoviePy] Writing video C:\Users\linas\projects\CarND-Advanced-Lane-Lines\results\project_video.mp4


100%|█████████▉| 1260/1261 [05:47<00:00,  3.63it/s]


[MoviePy] Done.
[MoviePy] >>>> Video ready: C:\Users\linas\projects\CarND-Advanced-Lane-Lines\results\project_video.mp4 

Wall time: 5min 47s


In [61]:
# Display the videos
video_in_folder  = os.path.join(root_folder, "data")
video_out_folder = os.path.join(root_folder, "results")
video_in_pattern = os.path.join(video_in_folder, "*.mp4")
video_in_fnames  = glob.glob(video_in_pattern)
video_out_fnames = [os.path.join(video_out_folder, os.path.basename(fname)) for fname in video_in_fnames]

video_index = 0
use_subclip = False

composed_out_folder = os.path.join(root_folder, "results", "intermediate")
composed_out_fname = os.path.join(composed_out_folder, "yellow_detection_" + os.path.basename(video_out_fnames[video_index]))

original_clip = VideoFileClip(video_in_fnames[video_index]).resize(0.5)
modified_clip = VideoFileClip(video_out_fnames[video_index]).resize(0.5)
if use_subclip:
    original_clip = original_clip.subclip(0, 10)
    modified_clip = modified_clip.subclip(0, 10)
composed_clip = clips_array([[original_clip, modified_clip]])

composed_clip.write_videofile(composed_out_fname, audio=False)


OSError: MoviePy error: the file C:\Users\linas\projects\CarND-Advanced-Lane-Lines\results\challenge_video.mp4 could not be found!
Please check that you entered the correct path.

In [90]:
original_clip.close()
modified_clip.close()
composed_clip.close()

## Auxiliary Tools
This should probably be split into a separate notebook, but I need some small tools to tinker with. E.g. I need to save one frame of an image to analyze the hue and lightness value of yellow line.

In [167]:
# video_in_name  = os.path.join(root_folder, "results", "project_video.mp4")
# video_in_name  = os.path.join(root_folder, "results", "harder_challenge_video.mp4")
video_in_name  = os.path.join(root_folder, "results", "intermediate", "channels", "s challenge_video.mp4")
out_folder = os.path.join(root_folder, "results", "intermediate", "dissected")

clip = VideoFileClip(video_in_name)

for i in range(int(clip.duration)):
    out_filename = os.path.join(out_folder, "frame_%d.png" % i)
    clip.save_frame(out_filename, t=i)
    
print("Done! Frames produced: %d" % clip.duration)
clip.close()

Done! Frames produced: 16
